# Chapter 3
## Discretization of a Convection-Diffusion Problem

We consider the equation
$$\begin{align}
-\mu\Delta u + v\cdot\nabla u &= f \quad \textrm{in } \Omega, \\
u &= g \quad \textrm{on } \partial\Omega.
\end{align}$$
The weak form is find $u_h\in V_h$ such that
$$\begin{equation}
\int_{\Omega}\mu\nabla u\nabla w dx + \int_{\Omega} v\cdot\nabla u w dx = \int_{\Omega} f w dx, \quad \forall w\in V_h,
\end{equation}$$
where $w$ are test functions. In one dimension on the unit interval with the boundary conditions $u(0) = 0$ and $u(1) = 1$, the problem has the analytical solution
$$u(x) = \frac{\exp{(-x/\mu)} - 1}{\exp{(-1/\mu)} - 1}.$$

## Exercise 3.1

Show that the matrix obtained from a central difference scheme applied to the operator $Lu = u_x$ is skew-symmetric. Furthermore, show that the matrix obtained by linear continuous Lagrange elements are also skew-symmetric. Remark: The matrix is only skew-symmetric in the interior of the domain, not at the boundary.

In [2]:
## Imports
import numpy   as np
import scipy   as sp
import dolfinx as dfx
import matplotlib.pyplot as plt


import ufl

from ufl      import grad, inner, div, dx
from mpi4py   import MPI
from petsc4py import PETSc

%matplotlib widget

In [2]:
## Exercise 3.1


## Exercise 3.2

Estimate numerically the constant in Cea’s lemma for various $\alpha$ and $h$ for the Example 3.1.

Cea's lemma states that
$$ \vert\vert u - u_h \vert\vert_V \leq C_1 h^t \vert\vert u \vert\vert_{t+1}$$
when solving the linear convection-diffusion problem with a finite element space approximation of order $t$. We denote the error $e_h = u - u_h$, and $h$ is the grid resolution. This puts the following bound on the constant $C_1$:
$$C_1 \geq \frac{\vert\vert u - u_h \vert\vert_V}{h^t \vert\vert u \vert\vert_{t+1}}$$.

In [31]:
## Exercise 3.2
plt.clf() # Clear previous figure
def left_boundary(x):
    ''' Locator function for the boundary at x = 0.
    '''
    return np.isclose(x[0], 0.0)

def right_boundary(x):
    ''' Locator function for the boundary at x = 1.
    '''
    return np.isclose(x[0], 1.0)

def calc_H1_error(mu, u_h):
    u_ex = du_analytical(mu)
    space = u_h.function_space
    u_ex_fun = dfx.fem.Function(space)
    u_ex_fun.interpolate(u_ex)
    du_h = u_h.dx(0)
    H1_error = dfx.fem.assemble_scalar(dfx.fem.form((u_ex_fun - du_h)**2 * dx))
    return H1_error

def calc_L2_error(mu, u_h):
    u_ex = u_analytical(mu)
    space = u_h.function_space
    u_ex_fun = dfx.fem.Function(space)
    u_ex_fun.interpolate(u_ex)
    L2_error = dfx.fem.assemble_scalar(dfx.fem.form((u_ex_fun - u_h) ** 2 * dx))
    return L2_error

class u_analytical:
    ''' Class for the function expression of the analytical solution.
    '''
    def __init__(self, mu):
        self.mu = mu
        
    def __call__(self, x):
        return (np.exp(-x[0] / self.mu) - 1) / (np.exp(-1 / self.mu) - 1)
    
class du_analytical:
    ''' Class for the function expression of the derivative of the analytical solution.
    '''
    def __init__(self, mu):
        self.mu = mu
        
    def __call__(self, x):
        return np.exp(-x[0] / self.mu) / (self.mu * (1 - np.exp(-1 / self.mu)))

fig_idx = 1
t = 1 # Order of polynomials in approximation
# Loop over different mesh resolutions
for mu_value in [1e-1, 1e-3, 1e-5]:
    #plt.figure(fig_idx)
    #plt.title(f'Mu = {mu_value}')
    print(f"Mu = {mu_value:g}\n")
    print("-----------------\n")
    for N in [10, 50, 100, 200, 400]:
        h = 1/N # Mesh resolution
        print(f"Mesh with h = {h}")
        mesh = dfx.mesh.create_unit_interval(MPI.COMM_WORLD, N) # Unit interval mesh
        P1 = ufl.FiniteElement('CG', mesh.ufl_cell(), 1) # Linear Lagrange elements
        V = dfx.fem.FunctionSpace(mesh, P1)

        u, w = ufl.TrialFunction(V), ufl.TestFunction(V) # Trial and test functions
        mu = dfx.fem.Constant(mesh, PETSc.ScalarType(mu_value)) # Viscosity
        f = dfx.fem.Constant(mesh, PETSc.ScalarType(0.0)) # Source term
        v = dfx.fem.Constant(mesh, PETSc.ScalarType(-1)) # Velocity

        # Bilinear and linear forms
        a = (mu*u.dx(0)*w.dx(0) + v * u.dx(0) * w) * dx
        L = f * w * dx

        bc1_value = dfx.fem.Constant(mesh, PETSc.ScalarType(0))
        bc2_value = dfx.fem.Constant(mesh, PETSc.ScalarType(1))

        bc1_entities = dfx.mesh.locate_entities_boundary(mesh, mesh.topology.dim - 1, left_boundary)
        bc1_dofs = dfx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, bc1_entities)
        bc1 = dfx.fem.dirichletbc(bc1_value, bc1_dofs, V)

        bc2_entities = dfx.mesh.locate_entities_boundary(mesh, mesh.topology.dim - 1, right_boundary)
        bc2_dofs = dfx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, bc2_entities)
        bc2 = dfx.fem.dirichletbc(bc2_value, bc2_dofs, V)

        bcs = [bc1, bc2]

        problem = dfx.fem.petsc.LinearProblem(a, L, bcs = bcs,
                                              petsc_options = {"ksp_type": "preonly", "pc_type": "lu"})
        u_h = problem.solve()
        #plt.plot(np.linspace(0, 1, N+1), u_h.x.array[:], label = f'{N} cells')
        H1_error = calc_H1_error(mu_value, u_h)
        L2_error = calc_L2_error(mu_value, u_h)
        print(f"H1 error norm: {H1_error:.2e}")
        print(f"L2 error norm: {L2_error:.2e}\n")
        
        # Estimate Cea's lemma constant
        u_L2_norm = dfx.fem.assemble_scalar(dfx.fem.form(u_h ** 2 * dx))
        C = H1_error/(h**t * u_L2_norm)
        print(f"Constant C: {C}\n")
        print("-----------------\n")
        
    
    #plt.legend()
    #plt.show()
    fig_idx += 1



Mu = 0.1

-----------------

Mesh with h = 0.1
H1 error norm: 4.01e-01
L2 error norm: 1.73e-04

Constant C: 4.697783805735971

-----------------

Mesh with h = 0.02
H1 error norm: 1.66e-02
L2 error norm: 2.77e-07

Constant C: 0.978568708617429

-----------------

Mesh with h = 0.01
H1 error norm: 4.17e-03
L2 error norm: 1.73e-08

Constant C: 0.48996343048597246

-----------------

Mesh with h = 0.005
H1 error norm: 1.04e-03
L2 error norm: 1.08e-09

Constant C: 0.24506696945511014

-----------------

Mesh with h = 0.0025
H1 error norm: 2.60e-04
L2 error norm: 6.76e-11

Constant C: 0.12254415311298912

-----------------

Mu = 0.001

-----------------

Mesh with h = 0.1
H1 error norm: 2.99e+04
L2 error norm: 6.37e+00

Constant C: 26463.70223240308

-----------------

Mesh with h = 0.02
H1 error norm: 5.35e+03
L2 error norm: 1.60e-02

Constant C: 263403.9828343739

-----------------

Mesh with h = 0.01
H1 error norm: 2.17e+03
L2 error norm: 3.56e-03

Constant C: 216103.00452999113

-------

## Exercise 3.3

Implement the convection-diffusion problem with $f = -\alpha u_{xx} - u_x$, where $u = \sin{(\pi x)}$.